In [ ]:

import dash
import core
from Scripts.actions import * 
from dash import html,dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate

# General Settings
# openai.api_type = "azure"
# openai.api_base = "https://chatbotv2.openai.azure.com/"
# openai.api_version = "2022-06-01-preview"
openai.api_key = os.getenv("OPENAI_API_KEY")


app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

divIntro= html.Div(children=[
    dcc.ConfirmDialog(
        id='connect-dialog',
        message="Changing Chatbot Options",
    ),
    dcc.ConfirmDialog(
        id='message-dialog',
    ),
    dbc.Container(fluid=True,className="py-3",
    children=[
        html.H1("MarBot", className="display-3"),
        html.P("This Chatbot is a proof of concept that demonstrates the use of GPT-3 and lower versions as new technologies.\
                It connects to Seeking Alpha and searches for trending news or analysis. The website provides two options for users,\
                Open question' and 'Summary', which allows users to ask questions or create summaries about the articles they choose.",className="lead")
                ],
            )], className="p-3 bg-light rounded-3",
)

divOptions = html.Div(
    dbc.Container( className="display-7", style={'margin-left':'5px'},children=[
        html.H3("Please Select One Option", style={'margin': '10px'}),
        # Chat Option
        html.Div(
            style= {'display':'flex'},
            children=[
                html.H4("ChatBot Mode", style={'margin': '12px'}),
                dbc.RadioItems(id="chatbot-mode",style = {'margin':'15px'}, options=[
                    {"label": "Open Questions", "value": 'OpenChatbot'},
                    {"label": "Smart  Summary", "value": 'Summary'}], inline=True)
                    ]
            ),

        html.Div(style= {'display':'flex'},children=[
            html.H4("Category", style={'margin': '12px'}),
            dbc.RadioItems(id="selected-mode",style = {'margin':'15px'}, options=[
                {"label": "Latest New", "value": 'latest-articles'},
                {"label": "Market News", "value": 'market-news'}], inline=True)]),

            # Div For Select
            html.Div(style={'display':'flex'}, children = [
                html.H4("Select Article", style={'margin': '12px', 'width':'30%'}),
                 dbc.Select(id="dropdown-article")]),

            ]),

       
    className="p-3 bg-light rounded-3",
    style={'border':'1px solid black', 'display': 'inline-block', 'width': '100%', 'vertical-align': 'top', "margin-top":'5px'}
)

divChat = html.Div(id = 'response-container', children=[])

app.layout = html.Div(style={'width': '100%', 'display': 'block', 'padding':'5px'}, children=[
    divIntro,
    html.Hr(style={'margin':'0px'}),
    divOptions,
    divChat])


# Connect2SeekingAlpha
@app.callback(
    Output("connect-dialog", 'displayed'),
    [Input('chatbot-mode', 'value')],
    [State('dropdown-article', 'options')],
)
def DownloadingArticles(value, options):
    if value is None:
        raise PreventUpdate
    else:
        core.TMP_CONTAINER = Connect2Seeking(trigger=True)
        return True

# Adding Article Selection 
@app.callback(
    Output("dropdown-article", 'options'),
    [Input('selected-mode','value')]
)
def UpdateList(mode:str):
    if mode is None:
        raise PreventUpdate
    else:
        tmp_options = []
        for name, _ in core.TMP_CONTAINER[mode].items():
            tmp_options.append({'label': name, 'value': name})
        return tmp_options

# Activate Chat Summary 
@app.callback(
    Output('response-container', 'children'),
    [Input('chatbot-mode','value')]
)
def UpdatingInterface (value:str):
    if value is None:
        raise PreventUpdate
    elif value == core.OPTIONS_MODE[0]:
        interface = html.Div(style={'display':'block'},children=[
            # ChatBot Answer Interface
            html.Div(id='chatbot-response', style={'min-height':'100px','border':'1px solid black','border-radius':'5px','margin-top':'5px'}, children = [
                html.Div(id='article-show', style={'padding': '10px', 'font-size': '18px'})
            ]),

            # User Interface
            html.Div(id='user-response', style={'display':'flex', 'margin-top':'5px'}, children = [
                dbc.Input(id='message', placeholder='Type your message here', type='text', style={'margin-right':'5px'}),
                html.Button("Submit", id="submit-button", className="btn btn-primary ml-2", n_clicks=0)]
                )
            ]
        )
        return interface
    elif value == core.OPTIONS_MODE[1]:
        interface = html.Div(className='d-flex',style={'margin-top': '10px'}, children=[
            html.Div(id='Summary-container', style={'min-width':'92%','min-height': '100px','border':'1px solid black','border-radius':'5px','margin-right':'5px'}),
            html.Button("Generate Summary", id="summary-button", className="btn btn-primary ml-2", n_clicks=0)])
        return interface



@app.callback(
    [Output('article-show', 'children')],
    [Input('selected-mode','value'),
     Input('dropdown-article','value')]
)

def ShowPartialNews (category, article):
    if any(criteria is None for criteria in [category, article]):
        raise PreventUpdate
    else:
        tmp_link_article = core.TMP_CONTAINER[category][article]
        tmp_article = GetArticle(Option=category, Selected=article, return_article=True)
        return [f"Partial Article : {tmp_article[:1000]} ... {tmp_article[-500:]} \n\nArticle Link:{tmp_link_article}"]
        # return html.Div(f"Partial Article : {tmp_article}\nEntire Article :{tmp_link_article}",style={'padding': '10px', 'font-size': '18px'})


@app.callback(
    [
        Output('message-dialog', 'message'),
        Output('message-dialog', 'displayed'),
        Output('Summary-container','children'),
    ],
    [Input('selected-mode','value'),
    Input('dropdown-article','value'),
    Input('chatbot-mode','value'),
    Input('summary-button', 'n_clicks')
    ]
)
def CreateSummary (Option,Article,Mode,n_clicks):

    if (n_clicks != 0):
        if (n_clicks < 3):
            print(Option,Article,Mode,n_clicks)
            if (Option != None) and (Article != None) and (Mode != None):
                Message, Summary = UserQuestion(Option,Article,Mode, Question = None)
                return Message, True, html.Div(Summary,style={'padding': '10px', 'font-size': '18px'})
            else:
                return  "One or more Options are empty", True, html.Div("Please Make sure Category and Article are Selected",style={'padding': '10px', 'font-size': '18px'})
        else:
            return "Only 3 Summaries, No more Credits", True, html.Div("Only 3 Summaries",style={'padding': '10px', 'font-size': '18px'})
    else:
        raise PreventUpdate


    


if __name__ == '__main__':
    app.run_server(debug=False)

In [ ]:
NavItem_style = {'display':'inline-flex'}
MarvinLinks = ["https://www.linkedin.com/in/marvin-garcia-917127184/","https://www.upwork.com/freelancers/~010d2ff47a39cfb4dd"]
GUIDANCE = "This Chatbot is a proof-of-concept that utilizes GPT-3 and lower versions to showcase the capabilities of new technologies. It connects to the financial news website Seeking Alpha and allows users to search for trending news and analysis. The bot offers two options for users - the ability to ask open-ended questions or generate summaries of the articles they select."
ERROR_MESSAGES = ["There was a Connection Error! Please Contact Marvin"]

In [1]:
import dash
import core
import time
from Scripts.actions import * 
from dash import html,dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Input, Output,State
from dash.exceptions import PreventUpdate

app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP], prevent_initial_callbacks=True)

# Container Nav and Instruction
DivNav = html.Div(children=[

    # Container Nav
    html.Div(dbc.NavbarSimple(brand="MarBot By Marvin G", brand_href=core.MarvinLinks[0],color="dark",dark=True, children = [
        dbc.NavItem(style=core.inline_flex, children =[
            dbc.NavLink("Linkedln", href=core.MarvinLinks[0]),
            dbc.NavLink("Upwork Profile",href =core.MarvinLinks[1])
        ])
    ])),

    # Instructions
    html.Div(className="p-3 bg-light rounded-3", children=[
        html.P(core.GUIDANCE, style=core.text_alignn_center, className="lead"),
        dcc.ConfirmDialog(id='popup',message=core.ERROR_MESSAGES[0])
    ])
])

# Container Options 
DivOption = html.Div(id='div-options', className="p-3 bg-light rounded-3", style=core.divoption_style, children=[

    # Main Accordion
    dbc.Accordion(style=core.width_40, children=[
        
        # Latest-Trend
        dbc.AccordionItem(title = "Step 1. Categories" , style=core.width_100,children =[
            html.Div(style= core.inline_flex,children=[
                html.H4("Options", style=core.margin_15),
                dbc.RadioItems(id="selected-type",style = core.margin_15, options=core.radio_article_options, inline=True)
            ])
        ]),

        # Articles
        dbc.AccordionItem(title="Step 2. Articles", style=core.width_100,children=[
            html.Div(style=core.block, children = [
                html.H4("Select Article", style=core.select_article_font_size),
                dcc.Loading(id='spinner',type="dot", children=[
                    dbc.Select(id="dropdown-article", children=[])
                ])
            ]),
        ]),

        # Chat Mode
        dbc.AccordionItem(title = "Step 3. ChatBot Mode" , style=core.width_100, children =[
            html.Div(style= core.block,children=[
                html.H4("MarBot Modes", style=core.margin_15),
                dbc.RadioItems(id="chatbot-mode",style = core.margin_15, inline=True, options=core.radio_mode_options)
            ])
        ]),
    ]),

    # Warning
    html.Div(style={'width':'100%'},children=[
            dbc.Card(style = core.margin_10,className="card-text",children=[
                dbc.CardBody( style = core.divcard_body, children=[
                    html.P(children=[
                        "If you're experiencing any issues with the website, try reloading the page.",
                        "Please note that this is just a proof of concept and there is a ",
                        html.Code('$50'),
                        " Credit limit for the OpenAI API.",
                        "Once the credit is used, the demo will be shut down."
                    ])
                ])
            ])
        ]),

    # Reset Button 
    html.Button("Reset", id="reset-button", className="btn btn-danger", style = {'margin-top':'10px','width':'100%'})
])

# Message Before Starting
DivChatSum = html.Div(style=core.width_100,children=[
    dbc.Card(style=core.height_100,className="card-text",children=[
        dbc.CardBody(id ="chatbot-response", children=[
            html.H5("Friendly Reminder" , id='reminder', className="card-title"),
            html.P(id='wait', children=["Wait , The Chatbot will ",html.Code( "answer") ," in here."]),
            dcc.Loading(id="summary-chat", type="dot", children=[])
        ])
    ])
])
        
# Middle Container
DivMid = html.Div(id='response', style=core.divmid_style, children=[DivOption,DivChatSum])

# Initiating Connection with SeekingAlpha 
@app.callback(
    [Output('dropdown-article', 'options'),
     Output('popup', 'displayed')],
    [Input('selected-type', 'value')]
)    

# Initiating Connection with SeekingAlpha 
def DownloadingArticles(value):
    """ Connect to SeekingAlpha
    Parameters;
        value: string, Latest new | market news
        
    """
    if value is None:
        raise PreventUpdate
    else:
        tmp_options = []
        core.TMP_CONTAINER = Connect2Seeking(trigger=True)
        
        
        if core.TMP_CONTAINER:
            for name, _ in core.TMP_CONTAINER[value].items():
                tmp_options.append({'label': name, 'value': name})
            time.sleep(2)
            return tmp_options , False
        else:
            time.sleep(2)
            return [] , True

# Loading Chatbot Summary
@app.callback(
    [Output('summary-chat','children'),
    Output('reminder','style'),
    Output('wait','style')],

    [Input('selected-type', 'value'),
     Input('dropdown-article','value'),
     Input('chatbot-mode', 'value')
     ]
)
def LoadingChatSpinningSummary(option_, article_ , mode_):
    if any(opt_ is None for opt_ in [article_ , mode_]):
        raise PreventUpdate
    else:

        Link = core.TMP_CONTAINER[option_][article_]
        _, Article_ = GetArticle(option_,article_)
        Article_ = Article_[list(Article_.keys())[0]][0]

        # Initating Connection to Openai 
        if mode_ == core.OPTIONS_MODE[0]: #OpenQuestion 
            # Cahtbot Interface
            DivChatbot = html.Div(id='chatbot-container', style=core.divchatbot_style,children=[
                html.Div(html.Img(src=core.IMAGE_LINK,style=core.divimg_style), id='chatbot-answer-start'),
                html.Div(children=[
                    core.CHABOT_PRE, 
                    html.H6(dbc.NavLink(f"Full Article in Here : {Link}", href=Link)),
                    html.Div(f"{Article_[:1000]}..."),
                    html.H6("Go Ahead ask me a question..!!!", style =core.margin_15)
                ])
            ])

            DivQuestion = html.Div(id='user-response', style=core.divuser_reponse, children = [
                dbc.Input(id='question', placeholder='Type your message here', type='text', style=core.margin_right_5),
                html.Button("Submit", id="submit-question", className="btn btn-primary ml-2", n_clicks=0)
            ])
            DivReponse = html.Div(id='message-container', children=[DivChatbot,DivQuestion])
            return DivReponse, core.display_none, core.display_none
        else:
            Message, Summary = UserQuestion(option_,article_,mode_, Question = None)
            time.sleep(2)
            HeadSummary = html.H5("MarBot Says", id='Summary', className="card-title")
            DivLink = html.H6(dbc.NavLink(f"Full Article in Here : {Link}", href=Link))
            DivHead1000 = html.H5(f"First 1000 Thousand Words - {article_}", id='Summary', className="card-title")
            DivF1000 = html.Div(f"{Article_[:1000]}... ", id='first-1000-words', style= core.text_alignn_justify)
            DivHeadSummary = html.H5("Smart Summary:", id='Head-Summary', className="card-title",style= core.margin_20)
            DivSummary = html.Div(Summary,style= core.text_alignn_justify)
            DivSummaryMain = html.Div(id='final-summary', children =[HeadSummary,DivLink,DivHead1000,DivF1000,DivHeadSummary,DivSummary])
            
            return DivSummaryMain, core.display_none, core.display_none

# Answering Questions
@app.callback(
    Output("message-container","children"),
    [Input("submit-question",'n_clicks')],
    [State('selected-type', 'value'),
     State('dropdown-article','value'),
     State('chatbot-mode', 'value'),
     State("question","value"),
     State("message-container","children")]
)
def QuestionAnswering(n_clicks, option_, article_ , mode_, question_, children):
    """ Validate Question and Connect To OpenAI"""

    if n_clicks == 0:
        raise PreventUpdate

    elif n_clicks == 4:
        return "Keep in mind that you are able to ask 3 questions only"
    
    elif n_clicks >= 1:
        _, Answer = UserQuestion(option_, article_ , mode_, question_.strip())
        DivChatbot = html.Div(id=f'chatbot-container-{n_clicks}', style=core.divchatbot_style,children=[
            children,
            html.Div(html.Img(src=core.IMAGE_LINK,style=core.divimg_style), id=f'chatbot-answer-{n_clicks}'),
            html.Div(f"Your Question : {question_}"),
            html.Div(html.H5("Answer", className="card-title",style= core.margin_20)),
            html.Div("here the Answer 123456789456123456788dafsfasfafas qtqeteqwtwetwet" , id=f"answer-1")
        ])
        
        return DivChatbot
    

app.layout = html.Div(children=[
    DivNav,
    DivMid])

if __name__ == '__main__':
    app.run_server(debug=False)

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:8050/ (Press CTRL+C to quit)
127.0.0.1 - - [21/Jan/2023 15:26:41] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2023 15:26:41] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2023 15:26:41] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2023 15:26:41] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2023 15:26:43] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [21/Jan/2023 15:26:47] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [21/Jan/2023 15:27:02] "POST /_dash-update-component HTTP/1.1" 204 -
127.0.0.1 - - [21/Jan/2023 15:27:25] "POST /_dash-update-component HTTP/1.1" 200 -


 Answer the question using the following context, and if the answer is not contained within the text below, say I don't know.

Context:
 in the third quarter caused a free cash flow deficit. The company needs to generate free cash flow as quickly as possible, not only to service that debt but also to pay it down. Any debt the company needs to refinance will likely result in higher interest payments, given how much interest rates have risen. There is about $13.6 billion in senior notes with a weighted average interest rate of 3.6% that will be due within the next five years. Additionally, the firm has $4.0 billion in short-term debts due in less than three years. They need to pay down some of their debt when it comes due because any debt that ends up being refinanced is likely to be more expensive. Conclusion The entertainment industry is evolving very fast and is currently characterized by increasing competition for subscribers and profitability. The high number of subscribers will eve

127.0.0.1 - - [21/Jan/2023 15:28:41] "POST /_dash-update-component HTTP/1.1" 200 -


 Answer the question using the following context, and if the answer is not contained within the text below, say I don't know.

Context:
 in the third quarter caused a free cash flow deficit. The company needs to generate free cash flow as quickly as possible, not only to service that debt but also to pay it down. Any debt the company needs to refinance will likely result in higher interest payments, given how much interest rates have risen. There is about $13.6 billion in senior notes with a weighted average interest rate of 3.6% that will be due within the next five years. Additionally, the firm has $4.0 billion in short-term debts due in less than three years. They need to pay down some of their debt when it comes due because any debt that ends up being refinanced is likely to be more expensive. Conclusion The entertainment industry is evolving very fast and is currently characterized by increasing competition for subscribers and profitability. The high number of subscribers will eve